#**Pipeline de Dados**
tem como objetivo realizar o processo de coleta, consolidação, tratamento e estruturação de dados públicos relacionados ao transporte rodoviário interestadual, com base em arquivos disponibilizados pela **Agência Nacional de Transportes Terrestres (ANTT).**

Os dados originais estavam distribuídos em múltiplos arquivos CSV, exigindo etapas de padronização, limpeza e organização para torná-los adequados para análise. Durante o desenvolvimento, foram aplicadas técnicas de tratamento utilizando a biblioteca Pandas, incluindo correção de valores nulos, padronização textual, criação de novas variáveis e reorganização do dataset.

Após o tratamento, os dados consolidados foram armazenados em um banco SQLite e utilizados na construção de dashboards no Power BI, permitindo análise visual e geração de insights a partir das informações estruturadas.




1_Conexão com o Google Drive

Nesta etapa foi realizada a conexão do ambiente do Google Colab com o Google Drive, permitindo acessar os arquivos CSV armazenados na pasta do projeto.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

2_Verificação dos Arquivos na Pasta

Aqui foi feita a listagem dos arquivos dentro da pasta do projeto para confirmar se os CSV estavam disponíveis para leitura.

In [ ]:
import os
os.listdir('/content/drive/MyDrive/Projeto ANTT')

3_Leitura e Consolidação dos Arquivos CSV

Nesta etapa:
* Foi definido o caminho da pasta
* Todos os arquivos .csv foram identificados
* Cada arquivo foi lido com pandas
* Foi criada a coluna origem_arquivo para rastreabilidade
* Todos os DataFrames foram unidos em um único DataFrame (df_total)

In [ ]:
import pandas as pd
import os

PASTA = "/content/drive/MyDrive/Projeto ANTT/Projeto"

dfs = []

for arquivo in os.listdir(PASTA):
    if arquivo.endswith(".csv"):
        caminho = os.path.join(PASTA, arquivo)
        df = pd.read_csv(caminho, encoding="latin-1", sep=";")
        df["origem_arquivo"] = arquivo
        dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)

4_Visualização Inicial dos Dados

Foi utilizada a função head() para visualizar as primeiras linhas do DataFrame consolidado.

In [ ]:
df_total.head(2)

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes,origem_arquivo
0,03/2019,02/2019,ALTO ARAGUAIA/MT,ITUIUTABA/MG,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",75.90,0.0,4,venda_passagem_03_2019.csv
1,03/2019,02/2019,ALTO GARCAS/MT,ARIQUEMES/RO,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",219.79,0.0,1,venda_passagem_03_2019.csv


5_Verificação de Valores Nulos

Aqui foi feita a contagem de valores nulos em cada coluna para identificar possíveis problemas de qualidade dos dados.

In [ ]:
print(df_total.isnull().sum())


mes_emissao_bilhete         0
mes_viagem                  0
ponto_origem_viagem         0
ponto_destino_viagem        0
tipo_servico            86124
tipo_gratuidade         40742
media_valor_total           0
dp_valor_total              0
quantidade_bilhetes         0
origem_arquivo              0
dtype: int64


6_Tratamento de Valores Nulos

Os valores ausentes nas colunas tipo_servico e tipo_gratuidade foram substituídos por "Não informado" para evitar inconsistências nas análises.

In [ ]:
df_total["tipo_servico"] = df_total["tipo_servico"].fillna("Não informado")
df_total["tipo_gratuidade"] = df_total["tipo_gratuidade"].fillna("Não informado")

In [ ]:
#Conferência para garantir que os valores nulos foram tratados corretamente.
print(df_total.isnull().sum())

mes_emissao_bilhete     0
mes_viagem              0
ponto_origem_viagem     0
ponto_destino_viagem    0
tipo_servico            0
tipo_gratuidade         0
media_valor_total       0
dp_valor_total          0
quantidade_bilhetes     0
origem_arquivo          0
dtype: int64


In [ ]:
df_total.head(2)

7_Criação da Coluna de Categoria de Desconto

Foi criada a coluna Desconto-gratuidade, extraindo apenas a parte anterior ao hífen da coluna tipo_gratuidade.

In [ ]:
df_total["Desconto-gratuidade"] = df_total["tipo_gratuidade"].str.split(" - ").str[0]

In [ ]:
df_total.head(2)

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes,origem_arquivo,Desconto-gratuidade
0,03/2019,02/2019,ALTO ARAGUAIA/MT,ITUIUTABA/MG,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",75.90,0.0,4,venda_passagem_03_2019.csv,Tarifa Promocional
1,03/2019,02/2019,ALTO GARCAS/MT,ARIQUEMES/RO,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",219.79,0.0,1,venda_passagem_03_2019.csv,Tarifa Promocional


8_Separação de Cidade e Estado

A coluna ponto_destino_viagem foi dividida em:
* cidade
* estado

Também foi feita padronização de capitalização.

In [ ]:
# Separando cidade e estado
df_total["cidade"] = df_total["ponto_destino_viagem"].str.split("/").str[0]
df_total["estado"] = df_total["ponto_destino_viagem"].str.split("/").str[1]

# Deixando a cidade em formato título (primeira letra maiúscula)
df_total["cidade"] = df_total["cidade"].str.title()

# Ajustando palavras comuns para minúsculo
palavras_minusculas = ["Da", "De", "Do", "Das", "Dos", "E"]
for palavra in palavras_minusculas:
    df_total["cidade"] = df_total["cidade"].str.replace(rf"\b{palavra}\b", palavra.lower(), regex=True)

# Criando coluna formatada
df_total["ponto_destino_viagem"] = df_total["cidade"] + " - " + df_total["estado"]

#########################################################################################################################

# Separando cidade e estado
df_total["cidade"] = df_total["ponto_origem_viagem"].str.split("/").str[0]
df_total["estado"] = df_total["ponto_origem_viagem"].str.split("/").str[1]

# Deixando a cidade em formato título (primeira letra maiúscula)
df_total["cidade"] = df_total["cidade"].str.title()

# Ajustando palavras comuns para minúsculo
palavras_minusculas = ["Da", "De", "Do", "Das", "Dos", "E"]
for palavra in palavras_minusculas:
    df_total["cidade"] = df_total["cidade"].str.replace(rf"\b{palavra}\b", palavra.lower(), regex=True)

# Criando coluna formatada
df_total["ponto_origem_viagem"] = df_total["cidade"] + " - " + df_total["estado"]

In [ ]:
df_total.head(2)

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes,origem_arquivo,Desconto-gratuidade,cidade,estado
0,03/2019,02/2019,Alto Araguaia - MT,Ituiutaba - MG,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",75.90,0.0,4,venda_passagem_03_2019.csv,Tarifa Promocional,Alto Araguaia,MT
1,03/2019,02/2019,Alto Garcas - MT,Ariquemes - RO,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",219.79,0.0,1,venda_passagem_03_2019.csv,Tarifa Promocional,Alto Garcas,MT


9_Criação do Dicionário de Estados

Foi criado um dicionário para converter siglas dos estados em nomes completos, facilitando análises e visualizações.

In [ ]:
# Dicionário de siglas para nomes completos
mapa_estados = {
    "AC": "Acre",
    "AL": "Alagoas",
    "AP": "Amapá",
    "AM": "Amazonas",
    "BA": "Bahia",
    "CE": "Ceará",
    "DF": "Distrito Federal",
    "ES": "Espírito Santo",
    "GO": "Goiás",
    "MA": "Maranhão",
    "MT": "Mato Grosso",
    "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais",
    "PA": "Pará",
    "PB": "Paraíba",
    "PR": "Paraná",
    "PE": "Pernambuco",
    "PI": "Piauí",
    "RJ": "Rio de Janeiro",
    "RN": "Rio Grande do Norte",
    "RS": "Rio Grande do Sul",
    "RO": "Rondônia",
    "RR": "Roraima",
    "SC": "Santa Catarina",
    "SP": "São Paulo",
    "SE": "Sergipe",
    "TO": "Tocantins",
    "ARG": "Argentina",
    "BOL": "Bolivia",
    "CHL": "Chile",
    "PER": "Peru",
    "PRY": "Paraguai",
    "URY": "Uruguai",
    "VEN": "Venezuela",
    "COL": "Colômbia"
}

# Supondo que você já separou a coluna "estado"
df_total["estado"] = df_total["estado"].map(mapa_estados)

In [ ]:
df_total.head(2)

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes,origem_arquivo,Desconto-gratuidade,cidade,estado
0,03/2019,02/2019,Alto Araguaia - MT,Ituiutaba - MG,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",75.90,0.0,4,venda_passagem_03_2019.csv,Tarifa Promocional,Alto Araguaia,Mato Grosso
1,03/2019,02/2019,Alto Garcas - MT,Ariquemes - RO,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",219.79,0.0,1,venda_passagem_03_2019.csv,Tarifa Promocional,Alto Garcas,Mato Grosso


10_Instalação da Biblioteca Unidecode

Instalação da biblioteca para remover acentuação e padronizar textos.

In [ ]:
!pip install unidecode

11_Normalização de Texto

Foi feita:
* Remoção de acentos
* Padronização de capitalização
* Separação e organização de cidade e estado

In [ ]:
from unidecode import unidecode

# Normalizar acentuação e capitalização
df_total["ponto_origem_viagem corrigido"] = df_total["ponto_origem_viagem"].apply(
    lambda x: f"{unidecode(x.split('-')[0]).title().strip()} - {unidecode(x.split('-')[1]).strip().upper()}"
    if "-" in x else unidecode(x).title().strip()
)

df_total["ponto_destino_viagem corrigido"] = df_total["ponto_destino_viagem"].apply(
    lambda x: f"{unidecode(x.split('-')[0]).title().strip()} - {unidecode(x.split('-')[1]).strip().upper()}"
    if "-" in x else unidecode(x).title().strip()
)

In [ ]:
df_total.head(2)

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes,origem_arquivo,Desconto-gratuidade,cidade,estado,ponto_origem_viagem corrigido,ponto_destino_viagem corrigido
0,03/2019,02/2019,Alto Araguaia - MT,Ituiutaba - MG,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",75.90,0.0,4,venda_passagem_03_2019.csv,Tarifa Promocional,Alto Araguaia,Mato Grosso,Alto Araguaia - MT,Ituiutaba - MG
1,03/2019,02/2019,Alto Garcas - MT,Ariquemes - RO,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",219.79,0.0,1,venda_passagem_03_2019.csv,Tarifa Promocional,Alto Garcas,Mato Grosso,Alto Garcas - MT,Ariquemes - RO


12_Renomeação das Colunas

As colunas foram renomeadas para nomes mais descritivos e amigáveis.

In [ ]:
df_total.rename(columns={
    "mes_emissao_bilhete": "Emissão do bilhete",
    "mes_viagem": "Mês da viagem",
    "ponto_origem_viagem": "ponto_origem_viagem",
    "ponto_destino_viagem": "ponto_destino_viagem",
    "tipo_servico": "Tipo de serviço",
    "tipo_gratuidade": "tipo_gratuidadee",
    "media_valor_total": "Média do valor final do bilhete",
    "dp_valor_total": "Dp valor final do bilhete",
    "quantidade_bilhetes": "Quantidade de bilhetes",
    "origem_arquivo":"Origem do arquivo",
    "Desconto-gratuidade": "Desconto-gratuidade",
    "cidade": "cidadee",
    "estado": "Estado de Origem",
    "ponto_origem_viagem corrigido": "Origem da viagem",
    "ponto_destino_viagem corrigido": "Destino da viagem"
}, inplace=True)

In [ ]:
df_total.head(2)

13_Remoção de Colunas Desnecessárias

Foram excluídas colunas intermediárias que não seriam usadas na análise final.

In [ ]:
#Excluir uma coluna específica
df_total.drop(columns=["ponto_origem_viagem"], inplace=True)
df_total.drop(columns=["ponto_destino_viagem"], inplace=True)
df_total.drop(columns=["tipo_gratuidadee"], inplace=True)
df_total.drop(columns=["Origem do arquivo"], inplace=True)
df_total.drop(columns=["cidadee"], inplace=True)

In [ ]:
df_total.head(2)

,Emissão do bilhete,Mês da viagem,Tipo de serviço,Média do valor final do bilhete,Dp valor final do bilhete,Quantidade de bilhetes,Desconto-gratuidade,Estado de Origem,Origem da viagem,Destino da viagem
0,03/2019,02/2019,Executivo,75.90,0.0,4,Tarifa Promocional,Mato Grosso,Alto Araguaia - MT,Ituiutaba - MG
1,03/2019,02/2019,Executivo,219.79,0.0,1,Tarifa Promocional,Mato Grosso,Alto Garcas - MT,Ariquemes - RO


14_Reorganização das Colunas

As colunas foram reorganizadas para melhorar a estrutura final do dataset.

In [ ]:
#Reordenar colunas
df_total = df_total[["Emissão do bilhete", "Mês da viagem", "Estado de Origem", "Origem da viagem","Destino da viagem",
"Tipo de serviço", "Desconto-gratuidade","Média do valor final do bilhete","Dp valor final do bilhete","Quantidade de bilhetes",]]

In [ ]:
df_total.head(2)

,Emissão do bilhete,Mês da viagem,Estado de Origem,Origem da viagem,Destino da viagem,Tipo de serviço,Desconto-gratuidade,Média do valor final do bilhete,Dp valor final do bilhete,Quantidade de bilhetes
0,03/2019,02/2019,Mato Grosso,Alto Araguaia - MT,Ituiutaba - MG,Executivo,Tarifa Promocional,75.90,0.0,4
1,03/2019,02/2019,Mato Grosso,Alto Garcas - MT,Ariquemes - RO,Executivo,Tarifa Promocional,219.79,0.0,1


15_Exportação para Banco SQLite

Os dados tratados foram exportados para um banco SQLite chamado Projeto_ANTT.db, criando a tabela viagens.

In [ ]:
import sqlite3

# Se não existir, ele cria o arquivo "viagens.db"
conexao = sqlite3.connect("Projeto_ANTT.db")

# Cria uma tabela chamada "viagens" e insere os dados
df_total.to_sql("viagens", conexao, if_exists="replace", index=False)

conexao.close()

16_Download do Banco de Dados

Permite baixar o arquivo .db para uso posterior.

In [ ]:
from google.colab import files
files.download("Projeto_ANTT.db")